## 0 Intro

本 note 聚焦于实现处理序列数据相关的神经网络功能层，具体包括：

- RNN - 处理单个数据的 RNN
- TimeRNN - 处理序列数据的 RNN
- LSTM - 处理单个数据的 LSTM
- TimeLSTM - 处理序列数据的 LSTM
- TimeAffine - 处理序列数据的 Affine
- TimeSoftmaxWithLoss - 处理序列数据的 SoftmaxWithLoss
- TimeDropout - 处理序列数据的 Dropout
- TimeBiLSTM - 处理序列数据的 BiLSTM

以上的 *Timexxx* 可以理解为在时间维度上展开的 *xxx*

## 01 RNN

RNN 层相比前馈神经网络中诸如 Affine 层，多了一个记忆上个隐状态的功能，因此可以处理序列数据，其计算图如下：



对比计算图，可以方便地实现 RNN 层的前向计算和后向计算

实现如下：

**成员变量**

- 参数 - `Wx`：输入数据对应的权重矩阵；`Wh`：上一次隐状态对应的权重矩阵；`b` 偏置向量
- 中间变量 - `x`：输入数据；`h_prev`：上一次隐状态；`h_next`：本次计算的隐状态
- 梯度 - `dWx` `dWh` `db` 以及返回前一层的导数 `dx`

**成员函数**

- `__init__(self, Wx, Wh, b)`
- `forward(self, x, h_prev)`
- `backward(self, dh_next)`

代码如下：

In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        '''三个参数均从外界传入，凸显了参数的初始化的重要性
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None  # 用于打包存储必须的中间结果
        
    def forward(self, x, h_prev):
        '''前向计算，RNN 需要上一个隐状态完成前向计算
        '''
        Wx, Wh, b = self.params
        t = np.dot(x, Wx) + np.dot(h_prev, Wh) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)  # 用元祖打包，避免无意修改
        return h_next